# Examples of usage of callbacks in Keras

- Save trained model and load it to continue training.


In [4]:
#First define a model
from keras.datasets import mnist
from keras.layers import Input, Dense
from keras.models import Model
from keras.utils import np_utils

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

batch_size = 64
nb_classes = 10
nb_epoch = 4


# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)



# Model
inputs = Input(shape=(784,))

x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(input=inputs, output=predictions)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# train the model
model.fit(X_train, Y_train,
          batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))



(60000, 'train samples')
(10000, 'test samples')
Train on 60000 samples, validate on 10000 samples
Epoch 1/4
60000/60000 [==============================] - 5s - loss: 0.3123 - acc: 0.9099 - val_loss: 0.1724 - val_acc: 0.9478
Epoch 2/4
60000/60000 [==============================] - 6s - loss: 0.1480 - acc: 0.9552 - val_loss: 0.1274 - val_acc: 0.9607
Epoch 3/4
60000/60000 [==============================] - 4s - loss: 0.1083 - acc: 0.9676 - val_loss: 0.1027 - val_acc: 0.9694
Epoch 4/4
60000/60000 [==============================] - 6s - loss: 0.0875 - acc: 0.9737 - val_loss: 0.0963 - val_acc: 0.9716


In [16]:
# Model checkpoint
from keras.callbacks import ModelCheckpoint
import pickle

model_path = '/tmp/keras/'

#Save model structure
print 'Saving architecture...'
json_string = model.to_json()
open(model_path + 'model_architecture.json', 'w').write(json_string)

#TRain and save checkpoints
print 'Train...'
checkpointer = ModelCheckpoint(filepath=model_path+"weights.{epoch:02d}-{val_acc:.5f}.hdf5", verbose=1)
history = model.fit(X_train, Y_train,
          batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test),
          callbacks=[checkpointer])

#Save history
pickle.dump( history.history, open( model_path + "history.p", "wb" ) )

print 'Done!'

Train on 60000 samples, validate on 10000 samples
Epoch 1/4
60000/60000 [==============================] - 4s - loss: 0.0128 - acc: 0.9962 - val_loss: 0.1477 - val_acc: 0.9732
Epoch 2/4
60000/60000 [==============================] - 5s - loss: 0.0122 - acc: 0.9966 - val_loss: 0.1376 - val_acc: 0.9765
Epoch 3/4
60000/60000 [==============================] - 7s - loss: 0.0107 - acc: 0.9969 - val_loss: 0.1348 - val_acc: 0.9758
Epoch 4/4
60000/60000 [==============================] - 4s - loss: 0.0102 - acc: 0.9969 - val_loss: 0.1435 - val_acc: 0.9761


In [23]:
for h in history.history.iteritems():
    print h

('acc', [0.99621666666666664, 0.99655000000000005, 0.99685000000000001, 0.99693333333333334])
('loss', [0.01282193367408181, 0.012193267197232004, 0.010729612696704983, 0.010205493709328585])
('val_acc', [0.97319999999999995, 0.97650000000000003, 0.9758, 0.97609999999999997])
('val_loss', [0.14774294006241434, 0.13756020433594826, 0.1348387435936805, 0.14346895182569375])


In [27]:
# Train starting on a previous weigths
from keras.models import model_from_json
from keras.optimizers import SGD

model3 = model_from_json(open(model_path + 'model_architecture.json').read())

model3.load_weights(model_path + 'weights.03-0.97610.hdf5')

model3.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
history2 = model3.fit(X_train, Y_train,
                      batch_size=batch_size, nb_epoch=nb_epoch,
                      validation_data=(X_test, Y_test))

Exception: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.